In [4]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from utils import prepare_dataframe
import pandas as pd
import numpy as np

In [5]:
train_df = pd.read_csv("./split-data/train_new.csv")

In [ ]:
val_df = pd.read_csv("./split-data/val_new.csv")

In [ ]:
test_df = pd.read_csv("./split-data/test_new.csv")

In [6]:
train_df.head()

,img_path,bbox,category,attributes
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,[ 72 79 232 273],Blouse,NaN
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,[ 67 59 155 161],Blouse,NaN
2,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,[ 51 62 167 182],Blouse,NaN
3,img/Sheer_Pleated-Front_Blouse/img_00000008.jpg,[ 75 74 197 246],Blouse,NaN
4,img/Sheer_Pleated-Front_Blouse/img_00000009.jpg,[ 24 56 149 235],Blouse,NaN


In [7]:
traingen=ImageDataGenerator(rescale= 1./255 , height_shift_range=0.2, rotation_range = 10,
                           horizontal_flip = True,zoom_range= [0.7,1.3])

In [9]:
train_generator=traingen.flow_from_dataframe(
dataframe=train_df,
directory="./dataset",
x_col="img_path",
y_col= "category",
batch_size=128,
shuffle= True,
target_size = (256,256))

Found 209222 validated image filenames belonging to 46 classes.


In [ ]:
valgen = ImageDataGenerator(rescale= 1./255)

In [10]:
val_generator=valgen.flow_from_dataframe(
dataframe=val_df,
directory="./dataset",
x_col="img_path",
y_col= "category",
batch_size=128,
shuffle= True,
target_size = (256,256))

Found 40000 validated image filenames belonging to 46 classes.


In [11]:
#To viualize data batches from Image Data Generator
x_batch, y_batch = next(train_generator)

#y_batch = np.expand_dims(y_batch,axis=-1)
print(x_batch.shape)
print(y_batch.shape)
print(type(x_batch))
print(type(y_batch))
print(type(x_batch[0]))
print(type(y_batch[0][0]))
print(y_batch)

'''
for i in range (0,32):
    image = x_batch[i]
    print(y_batch[i][0].shape)
    plt.imshow(image.astype(np.uint8))
    plt.show()
'''

(128, 256, 256, 3)
(128, 46)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float32'>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


'\nfor i in range (0,32):\n    image = x_batch[i]\n    print(y_batch[i][0].shape)\n    plt.imshow(image.astype(np.uint8))\n    plt.show()\n'

PREPARING MODEL

In [12]:
from keras.models import Model
from keras.layers import Dense
from keras.regularizers import l2
from keras.optimizers import SGD,Adam
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import DirectoryIterator, ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard
from keras import backend as K

In [13]:
import tensorflow as tf

In [14]:
model_resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape = (256,256,3))

In [15]:
#Setting every layer except the last 3 layers as non trainable
for layer in model_resnet.layers[:-3]:
    layer.trainable = False

In [16]:
#Branch for category classification
#We build a model in top of the resnet-50 model
x = model_resnet.output
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
y = Dense(46, activation='softmax', name='img')(x)

In [ ]:
'''
#Branch for Bounding Box prediction
x_bbox = model_resnet.output
x_bbox = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)
x_bbox = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)
bbox = Dense(4, kernel_initializer='normal', name='bbox')(x_bbox)
'''

In [17]:
final_model = Model(inputs= model_resnet.input, outputs=y)

In [18]:
print(final_model.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [19]:
opt = Adam(lr=0.0001, name = "Adam")

In [20]:
#Compiling the final model
final_model.compile(optimizer=opt,
                    loss={'img': 'categorical_crossentropy'},
                          #'bbox': 'mean_squared_error'},
                    metrics={'img': ['accuracy', 'top_k_categorical_accuracy']}) # default: top-5
                           #  'bbox': ['mse']})

In [29]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               patience=12,
                               factor=0.5,
                               verbose=1)
tensorboard = TensorBoard(log_dir='./logs')
early_stopper = EarlyStopping(monitor='val_loss',
                              patience=30,
                              verbose=1)
checkpoint = ModelCheckpoint('./models/model.h5')

In [21]:
def custom_generator(iterator):
    while True:
        batch_x, batch_y = next(iterator)
        batch_y = np.expand_dims(y_batch,axis=-1)
        yield (batch_x,batch_y)

In [23]:
#Fitting the model on the training sata
final_model.fit(custom_generator(train_generator),steps_per_epoch=20, epochs=1, 
                validation_data = custom_generator(val_generator), validation_steps=32)

20/20 [==============================] - 462s 23s/step - loss: 3.9675 - accuracy: 0.1703 - top_k_categorical_accuracy: 0.4895 - val_loss: 3.4509 - val_accuracy: 0.2019 - val_top_k_categorical_accuracy: 0.6331


In [26]:
testgen = ImageDataGenerator()

test_generator=testgen.flow_from_dataframe(
dataframe=test_df,
directory="./dataset",
x_col="img_path",
y_col= "category",
batch_size=128,
shuffle= True,
target_size = (256,256))

Found 40000 validated image filenames belonging to 46 classes.


In [27]:
results = final_model.evaluate(custom_generator(test_generator), steps=200)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [29]:
# serialize model to JSON
model_json = final_model.to_json()
with open("./models/model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
final_model.save_weights("./models/model.h5")
print("Saved model to disk")

Saved model to disk


In [31]:
print(results)

[4.113183498382568, 0.036445312201976776, 0.35445311665534973]


In [ ]:
'''
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
'''

In [33]:
print('Image loss: ' + str(results[0]))
print('Image accuracy: ' + str(results[1]))
print('Top-5 image accuracy: ' + str(results[2]))

Image loss: 4.113183498382568
Image accuracy: 0.036445312201976776
Top-5 image accuracy: 0.35445311665534973
